In [ ]:
import numpy as np
import scipy.io.wavfile
import matplotlib.pyplot as plt
from time import time 

In [ ]:
"""
Preparare data

You'll need to download the data from
https://databricks.com/blog/2019/04/30/understanding-dynamic-time-warping.html, in particular:

- doors-and-corners-kid_thats-where-they-get-you.wav
- doors-and-corners-kid_thats-where-they-get-you-2.wav
"""

audio_path = '/Users/kunigami/Downloads/audio_samples/'


filename1 = f'{audio_path}/doors-and-corners-kid_thats-where-they-get-you.wav'
filename2 = f'{audio_path}/doors-and-corners-kid_thats-where-they-get-you-2.wav'

[sample_rate1, amplitudes1] = scipy.io.wavfile.read(filename1)
amplitudes1 = np.array(amplitudes1)
# single channel
amplitudes1 = amplitudes1[:, 0]

[sample_rate2, amplitudes2] = scipy.io.wavfile.read(filename2)
amplitudes2 = np.array(amplitudes2)
# single channel
amplitudes2 = amplitudes2[:, 0]

assert sample_rate1 == sample_rate2, "should have same sample rate"

print(f'Original sizes: {len(amplitudes1)} {len(amplitudes2)}')

fig, axs = plt.subplots(3, 1, figsize=(18, 5))

# Manually chosen offsets to make the series match on their start
s1 = 28000
e1 = 29195
samples1 = amplitudes1[s1:e1]
samples1_1000 = amplitudes1[s1:s1 + 1000]
samples1_10000 = amplitudes1[s1:s1 + 10000]
xs1 = range(len(samples1))
axs[0].plot(xs1, samples1)

s2 = 63750
e2 = 64850
samples2 = amplitudes2[s2:e2]
samples2_1000 = amplitudes1[s2:s2 + 1000]
samples2_10000 = amplitudes1[s2:s2 + 10000]
xs2 = range(len(samples2))
axs[1].plot(xs2, samples2)

s3 = 40000
e3 = 41000
samples3 = amplitudes1[s3:e3]
xs3 = range(len(samples3))
axs[2].plot(xs3, samples3)


In [ ]:
"""
Simple Dynamic Time Wrapping, O(nm) running time, O(nm) space.
"""
def path_from_opt(opt):
    i = len(opt) - 1
    j = len(opt[-1]) - 1

    # note that opt's range are shifted by 1, so we
    # need to subtract 1 to obtain xs's and ys's 
    # indexes
    path = [[i - 1, j - 1]]
    while i > 0:                       
        if (opt[i - 1, j - 1] < opt[i - 1, j] and  
            opt[i - 1, j - 1] < opt[i, j - 1]):
            i, j = i - 1, j - 1
        elif opt[i - 1, j] < opt[i, j - 1]:
            i = i - 1
        else:
            j = j - 1
        
        if i > 0 and j > 0:
            path.append([i - 1, j - 1])
    
    # reverse backtrack order
    return path[::-1]

def dtw(xs, ys):
    n, m = len(xs), len(ys)
    
    opt = np.full((n+1, m+1), np.inf)
    opt[0, 0] = 0

    for i in range(1, n+1):
        for j in range(1, m+1):            
            min_dist = min([opt[i-1, j], opt[i, j-1], opt[i-1, j-1]])
            opt[i, j] = abs(xs[i-1] - ys[j-1]) + min_dist

    return opt

def plot_matching():
    fig, ax = plt.subplots(1, 1, figsize=(18, 5))
    xs1 = range(len(xs))
    ax.plot(xs1, xs)

    xs2 = range(len(ys))
    ax.plot(xs2, ys)

    linestyle = (0, (1, 1))
    for [xid, yid] in path:
        p1 = (xid, xs[xid])
        p2 = (yid, ys[yid])
        plt.plot([xid, yid], [xs[xid], ys[yid]], color='r', linestyle=linestyle)
        
xs = samples1
ys = samples2

opt = dtw(xs, ys)
print('Distance: ', opt[-1][-1])

path = path_from_opt(opt)
plot_matching(path)


In [ ]:
"""
Implementation of FastDTW and comparison with other methods.
"""

def get_absolute_j(i, j, wrange):
    if wrange is None:
        return j
    return min(wrange[i][0] + j, wrange[i][1])

def get_relative_j(i, j, wrange):
    if wrange is None:
        return j    
    return max(j - wrange[i][0], 0) 

def get_neighbors(i, abs_j, opt, wrange):
    neighbors = []
    for [di, dj] in [[-1, -1], [-1, 0], [0, -1]]:
        i2 = i + di
        j2 = get_relative_j(i2, abs_j, wrange) + dj
        if j2 >= 0 and j2 < len(opt[i2]):
            neighbors.append([i2, j2])
    return neighbors

def path_from_windowed_opt(opt, wrange = None, debug=False):
    i = len(opt) - 1
    # j is relative to the windowed row
    j = len(opt[-1]) - 1
    
    path = []
        
    while i > 0:
        
        abs_j = get_absolute_j(i, j, wrange)
        
        if debug:
            print(f'adding: [{i - 1},{abs_j}]')

        path.append([i - 1, abs_j])
     
        neighbors = get_neighbors(i, abs_j, opt, wrange)
                    
        min_value = min(
            [opt[i2][j2] for [i2, j2] in neighbors]
        )
        
        # set i,j to the first neighbor that yields the optimal solution
        [i, j] = next(
            [i2, j2] for [i2, j2] in neighbors \
                if opt[i2][j2] == min_value
        )
                
    # reverse backtrack order
    return path[::-1]

def windowed_dtw(xs, ys, wrange = None):
    
    n, m = len(xs), len(ys)
    
    if wrange is not None:
        assert len(wrange) == n + 1

    opt = []
    for i in range(n + 1):
        jmin, jmax = wrange[i] if wrange else [0, m - 1]            
        opt.append(np.full(jmax - jmin + 1, np.inf))

    for i in range(1, n + 1):
        for j in range(len(opt[i])):
                        
            abs_j = get_absolute_j(i, j, wrange)
            
            # special case: value = 0 for empty series
            if i == 1 and abs_j == 0:
                opt[i][j] = abs(xs[i - 1] - ys[abs_j])
            else:
                prev_j = get_relative_j(i - 1, abs_j, wrange)

                # valid adjacent cells
                neighbors = get_neighbors(i, abs_j, opt, wrange)

                min_value = min(
                    [opt[i2][j2] for [i2, j2] in neighbors]
                )

                opt[i][j] = min_value + abs(xs[i - 1] - ys[abs_j])
            
    return opt

def get_search_window(path, n, m, radius):
    
    # initialize with empty ranges
    wrange = [None]*(n + 1)
    wrange[0] = [0, m - 1]
    
    def add_range(i, jmin, jmax):
        w = wrange[i]
        if w is not None:
            jmin = min(w[0], jmin)
            jmax = max(w[1], jmax)
        wrange[i] = [clamp(jmin, 0, m-1), clamp(jmax, 0, m-1)]
            
    for i, j in path:
        imin, imax = 2*i, 2*i + 1 
        
        jmin, jmax = 2*j - radius, 2*j + 1 + radius
        add_range(imin, jmin, jmax)
        
        if imax < n:
            add_range(imax, jmin, jmax)            
            
    # fill in ranges that were not set with defaults
    for i in range(len(wrange)):
        if wrange[i] is None:
            wrange[i] = [0, m - 1]
            
    return wrange

def halve(xs):
    new_xs = []
    n2 = len(xs)//2
    for i in range(n2):
        new_xs.append((xs[2*i] + xs[2*i+1])/2)
    
    # odd number of points
    if len(xs) % 2 == 1:
        new_xs.append(xs[-1])
     
    return new_xs

def fast_dtw(xs, ys, radius = 30):
    min_size = radius + 2
    if len(xs) <= min_size or len(ys) <= min_size:
        opt = windowed_dtw(xs, ys)
        return path_from_windowed_opt(opt)

    lowres_xs = halve(xs)
    lowres_ys = halve(ys)
    
    lowres_path = fast_dtw(lowres_xs, lowres_ys, radius)
        
    wrange = get_search_window(lowres_path, len(xs), len(ys), radius)
    
    opt = windowed_dtw(xs, ys, wrange)
    return path_from_windowed_opt(opt, wrange)
    
def compute_cost_from_path(xs, ys, path):
    cost = 0
    for [i, j] in path:
        cost += abs(xs[i] - ys[j])
    return cost

def clamp(value, low, high):
    return min(max(value, low), high)

def pretty_print(opt, m, wrange=None):
    r = '{0: <3}'.format('*')
    for j in range(m):
        r += '{0: <7}'.format(j)
    print(r)

    for i in range(len(opt)):
        if wrange:
            jmin, jmax = wrange[i]
        else:
            jmin, jmax = 0, m-1

        r = '{0: <3}'.format(i)
        for j in range(len(ys)):
            if j < jmin or j > jmax: 
                r += '{0: <7}'.format('-')
            else:
                r += '{0: <7}'.format(opt[i][j - jmin])
        print(r)

def get_window_range(xs, ys, w):
    wrange = [[0, len(ys) - 1]]
    for i in range(len(xs)):
        wrange.append([clamp(i - w, 0, len(ys) - 1), clamp(i + w, 0, len(ys) - 1)])
    return wrange

xs = samples1
ys = samples2

"""
Vanilla DTW
"""

start = time()
truth_opt = dtw(xs, ys)
truth = truth_opt[-1][-1]
duration = int(time() - start)

print(f'ground truth {dtw_value(xs, ys)} [{duration}s]')

"""
No window, but uses the dtw_window_range() function
"""

start = time()
opt = dtw_window_range(xs, ys)

path = path_from_opt(opt)

cost = compute_cost_from_path(xs, ys, path)
duration = int(time() - start)

assert cost == opt[-1][-1]

print(f'no window {cost} [{duration}s]')

"""
Fixed window
"""

wrange = get_window_range(xs, ys, 5)
start = time()
opt = dtw_window_range(xs, ys, wrange)

path = path_from_opt(opt, wrange, False)
cost = compute_cost_from_path(xs, ys, path)
duration = int(time() - start)

assert cost == opt[-1][-1]

print(f'windowed {cost} [{duration}s]')

"""
FastDTW
"""

start = time()
path = fast_dtw(xs, ys)
approx = compute_cost_from_path(xs, ys, path)
duration = int(time() - start)

print(f'fast dtw: {approx} [{duration}s]')
print('Error: {:.2f}%'.format(abs(approx - truth)/truth * 100))


In [ ]:
"""Experimentation: Performance"""

def get_fixed_window(xs, ys, w):
    wrange = []
    w = max(w//2, abs(len(ys) - len(xs)) + 1)    
    for i in range(len(xs) + 1):
        jmin = max(i - w, 0)
        jmax = min(jmin + 2*w, len(ys) - 1)
        wrange.append([jmin, jmax])
    return wrange

def run_multiple(f, n = 10):
    start = time()
    for i in range(n):
        f()
    duration = time() - start
    return duration / n

def series_dtw():
    xs = samples1
    ys = samples2
    dtw(xs, ys)

def series_fastdtw():
    xs = samples1
    ys = samples2
    fast_dtw(xs, ys)

def series10000_fastdtw():
    xs = samples1_10000
    ys = samples2_10000
    fast_dtw(xs, ys)

def series_fixed_window():
    xs = samples1
    ys = samples2

    wrange = get_fixed_window(xs, ys, 100)
    return dtw_window_range(xs, ys, wrange)[-1][-1]

def series10000_fixed_window():
    xs = samples1_10000
    ys = samples2_10000

    wrange = get_fixed_window(xs, ys, 100)
    return dtw_window_range(xs, ys, wrange)[-1][-1]

experiments = {
#     'series_dtw': series_dtw,
#     'series_fastdtw': series_fastdtw,
#     'series10000_fastdtw': series10000_fastdtw,
    'series_fixed_window': series_fixed_window,
    'series10000_fixed_window': series10000_fixed_window
}
    
for exp_name in experiments.keys():
    experiment = experiments[exp_name]
    duration = run_multiple(experiment)
    print('{0: <30} {1:.4f}'.format(exp_name, duration))

In [ ]:
"""Experimentation: Precision"""

def series_dtw():
    xs = samples1
    ys = samples2
    return dtw(xs, ys)[-1][-1]
    
def series_fastdtw():
    xs = samples1
    ys = samples2
    return compute_cost_from_path(xs, ys, fast_dtw(xs, ys))

def series_fixed_window():
    wrange = get_fixed_window(xs, ys, 100)
    return dtw_window_range(xs, ys, wrange)[-1][-1]

def series10000_fastdtw():
    xs = samples1_10000
    ys = samples2_10000
    return compute_cost_from_path(xs, ys, fast_dtw(xs, ys))
 
def series10000_fixed_window():
    xs = samples1_10000
    ys = samples2_10000

    wrange = get_fixed_window(xs, ys, 100)
    return dtw_window_range(xs, ys, wrange)[-1][-1]

experiments = {
    'series_dtw': series_dtw,
    'series_fastdtw': series_fastdtw,
    'series_fixed_window': series_fixed_window,
    'series10000_fastdtw': series10000_fastdtw,
    'series10000_fixed_window': series10000_fixed_window
}

ground = series_dtw()

for exp_name in experiments.keys():
    experiment = experiments[exp_name]
    cost = experiment()
    p = (cost - ground)/ground * 100
    print('{0: <25} {1: <8} {2:.2f}%'.format(exp_name, int(cost), p))

In [ ]:
"""Experimentation: Non-similar series"""

def get_similar_series():
    return [samples1, samples2]

def get_diff_series():
    return [samples1, samples3]

experiments = {
    'same series': get_similar_series,
    'diff series': get_diff_series,
}

for exp_name in experiments.keys():
    get_series = experiments[exp_name]
    [xs, ys] = get_series()
    cost = dtw(xs, ys)[-1][-1]
    xnorm_cost = cost / (len(xs) + len(ys))
    
    p = (cost - ground)/ground * 100
    print('{0: <20} {1: <8} {2: <8}'.format(exp_name, int(cost), int(xnorm_cost)))